In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import folium as fol
import pickle

%matplotlib inline

In [10]:
import pickle

airports_loc_pairs = pd.read_pickle('data/airport_coordinates.pickle')

airports_loc = pd.DataFrame([[iata, loc[0], loc[1]] for iata, loc in airports_loc_pairs.items() if loc], columns=['Code', 'Lat', 'Lng'])
airports_loc = airports_loc.set_index('Code')
airports_loc

airlines = pd.read_csv('data/L_AIRLINE_ID.csv', index_col='Code')
airports = pd.read_csv('data/L_AIRPORT.csv', index_col='Code')

airlines_splitted = airlines.Description.apply(lambda d: d.split(': '))
airlines['CarrierName'] = airlines_splitted.apply(lambda d: d[0])
airlines['CarrierCode'] = airlines_splitted.apply(lambda d: d[-1])
airlines = airlines.drop('Description', axis=1)

airports_splitted = airports.Description.apply(lambda d: d.split(': '))
airports['AirportLocation'] = airports_splitted.apply(lambda d: d[0])
airports['AirportName'] = airports_splitted.apply(lambda d: d[-1])
airports = airports.merge(airports_loc, how='left', left_index=True, right_index=True)
airports = airports.drop('Description', axis=1)


def getMonthlyDelays(month,year):
    data = pd.read_csv('data/On_Time_On_Time_Performance_2017_1.csv')
    data = data.merge(airlines, how='left', left_on='AirlineID', right_on='Code', right_index=True)
    #data = data[[
    #    'Year', 'Month', 'DayofMonth', 'DayOfWeek',
    #    'CarrierName', 'CarrierCode', 'FlightNum', 'TailNum',
    #    'Origin', 'Dest',
    #    'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime', 'ArrDelay',
    #    'CRSElapsedTime', 'ActualElapsedTime', 'Distance',
    #    'Cancelled', 'CancellationCode', 'Diverted',
    #    'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'
    #]]
    data['Flights'] = 1

    #airports_delay = data[['DepDelayMinutes', 'Origin']].groupby('Origin').quantile(q=0.9)
    #airports_flights = data[['Flights', 'Origin']].groupby('Origin').sum()
    #airports_delay = airports_delay.merge(airports_flights, how='left', left_index=True, right_index=True)
    #airports_delay = airports_delay.merge(airports_loc, how='left', left_index=True, right_index=True)
    ##airports_delay.to_csv('airports-delay.csv')
    
    airport_delay_causes = data[['Origin', 'DayofMonth', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']].copy()
    airport_delay_causes['UnknownDelay'] = -data['DepDelayMinutes']
    airport_delay_causes = airport_delay_causes.set_index(['Origin', 'DayofMonth'])
    airport_delay_causes = airport_delay_causes.fillna(0)
    airport_delay_causes = airport_delay_causes.groupby(['Origin', 'DayofMonth']).sum()
    airport_delay_causes['UnknownDelay'] = -airport_delay_causes.sum(axis=1)
    airport_delay_causes['UnknownDelay'][airport_delay_causes['UnknownDelay'] < 0] = 0
    airport_delay_causes = airport_delay_causes.sort_index()

    return airport_delay_causes

In [17]:
import time
import os

date_range = []
for year in range(1987, 2018):
    for month in range(1, 13):
        if not (year == 2017 and month >= 10):
            date_range.append({'year':year,'month':month})


for date in date_range:
    dirname = "delays/{year}/{month}/".format(**date)
    os.makedirs(dirname, exist_ok=True)
    start = time.time()
    mon = getMonthlyDelays(**date)
    mon.to_csv(dirname+'airport_causes.csv'.format(**date))
    print("Monthly delays for {month}-{year} done ({seconds})".format(seconds=time.time()-start, **date))


/home/cranium/.anaconda3/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Monthly delays for 1-1987 done (3.3743104934692383)
Monthly delays for 2-1987 done (3.3270888328552246)
Monthly delays for 3-1987 done (3.308990240097046)
Monthly delays for 4-1987 done (3.1677727699279785)
Monthly delays for 5-1987 done (3.31842041015625)
Monthly delays for 6-1987 done (3.2278389930725098)
Monthly delays for 7-1987 done (3.457570791244507)
Monthly delays for 8-1987 done (3.399739980697632)
Monthly delays for 9-1987 done (3.460057020187378)
Monthly delays for 10-1987 done (3.3790247440338135)
Monthly delays for 11-1987 done (3.2348861694335938)
Monthly delays for 12-1987 done (3.272721767425537)
Monthly delays for 1-1988 done (3.3051557540893555)
Monthly delays for 2-1988 done (3.2428600788116455)
Monthly delays for 3-1988 done (3.2444212436676025)
Monthly delays for 4-1988 done (3.354300022125244)
Monthly delays for 5-1988 done (3.5912845134735107)
Monthly delays for 6-1988 done (3.233557939529419)
Monthly delays for 7-1988 done (3.3826799392700195)
Monthly delays for

Monthly delays for 4-2000 done (3.269148588180542)
Monthly delays for 5-2000 done (3.4215261936187744)
Monthly delays for 6-2000 done (3.553457021713257)
Monthly delays for 7-2000 done (3.178581953048706)
Monthly delays for 8-2000 done (3.11979341506958)
Monthly delays for 9-2000 done (3.1757121086120605)
Monthly delays for 10-2000 done (3.2317044734954834)
Monthly delays for 11-2000 done (3.3572514057159424)
Monthly delays for 12-2000 done (3.2062838077545166)
Monthly delays for 1-2001 done (3.2488553524017334)
Monthly delays for 2-2001 done (3.503779172897339)
Monthly delays for 3-2001 done (3.28648042678833)
Monthly delays for 4-2001 done (3.2259278297424316)
Monthly delays for 5-2001 done (3.2621982097625732)
Monthly delays for 6-2001 done (3.25985050201416)
Monthly delays for 7-2001 done (3.3549551963806152)
Monthly delays for 8-2001 done (3.318769931793213)
Monthly delays for 9-2001 done (3.4429030418395996)
Monthly delays for 10-2001 done (3.227583169937134)
Monthly delays for 1

Monthly delays for 7-2013 done (3.239116907119751)
Monthly delays for 8-2013 done (3.2796528339385986)
Monthly delays for 9-2013 done (3.2371718883514404)
Monthly delays for 10-2013 done (3.2715811729431152)
Monthly delays for 11-2013 done (3.32541823387146)
Monthly delays for 12-2013 done (3.2405755519866943)
Monthly delays for 1-2014 done (3.7589330673217773)
Monthly delays for 2-2014 done (3.6578948497772217)
Monthly delays for 3-2014 done (3.516664505004883)
Monthly delays for 4-2014 done (3.522472858428955)
Monthly delays for 5-2014 done (3.180440664291382)
Monthly delays for 6-2014 done (3.7094719409942627)
Monthly delays for 7-2014 done (3.419642448425293)
Monthly delays for 8-2014 done (3.397170066833496)
Monthly delays for 9-2014 done (3.130364418029785)
Monthly delays for 10-2014 done (3.199307918548584)
Monthly delays for 11-2014 done (3.2078323364257812)
Monthly delays for 12-2014 done (3.1506524085998535)
Monthly delays for 1-2015 done (3.2418203353881836)
Monthly delays f

In [35]:
print("Read data and filter columns")
data = pd.read_csv('data/On_Time_On_Time_Performance_2017_1.zip')
data = data.merge(airlines, how='left', left_on='AirlineID', right_on='Code', right_index=True)
print("# of columns before: {}".format(data.columns.size))
data = data[[
    'Year', 'Month', 'DayofMonth', 'DayOfWeek',
    'CarrierName', 'CarrierCode', 'FlightNum', 'TailNum',
    'Origin', 'Dest',
    'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime', 'ArrDelay',
    'CRSElapsedTime', 'ActualElapsedTime', 'Distance',
    'Cancelled', 'CancellationCode', 'Diverted',
    'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'
]]
data['Flights'] = 1
print("# of columns  after: {}".format(data.columns.size))
data

Read data and filter columns


/home/thierry/bin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# of columns before: 112
# of columns  after: 28


,Year,Month,DayofMonth,DayOfWeek,CarrierName,CarrierCode,FlightNum,TailNum,Origin,Dest,...,Distance,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Flights
0,2017,1,17,2,American Airlines Inc.,AA,494,N583AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
1,2017,1,18,3,American Airlines Inc.,AA,494,N544AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
2,2017,1,19,4,American Airlines Inc.,AA,494,N553AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
3,2017,1,20,5,American Airlines Inc.,AA,494,N191AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,33.0,0.0,0.0,0.0,0.0,1
4,2017,1,21,6,American Airlines Inc.,AA,494,N170AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
5,2017,1,22,7,American Airlines Inc.,AA,494,N179AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,0.0,0.0,8.0,0.0,17.0,1
6,2017,1,23,1,American Airlines Inc.,AA,494,N579AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
7,2017,1,24,2,American Airlines Inc.,AA,494,N583AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
8,2017,1,25,3,American Airlines Inc.,AA,494,N167AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
9,2017,1,26,4,American Airlines Inc.,AA,494,N551AA,CLT,PHX,...,1773.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1


In [37]:
import pickle

airports_loc_pairs = pd.read_pickle('data/airport_coordinates.pickle')
airports_loc = pd.DataFrame([[iata, loc[0], loc[1]] for iata, loc in airports_loc_pairs.items() if loc], columns=['Code', 'Lat', 'Lng'])
airports_loc = airports_loc.set_index('Code')
airports_loc

airports_delay = data[['DepDelayMinutes', 'Origin']].groupby('Origin').quantile(q=0.9)
airports_flights = data[['Flights', 'Origin']].groupby('Origin').sum()
airports_delay = airports_delay.merge(airports_flights, how='left', left_index=True, right_index=True)
airports_delay = airports_delay.merge(airports_loc, how='left', left_index=True, right_index=True)
airports_delay.to_csv('airports-delay.csv')
airports_delay

,DepDelayMinutes,Flights,Lat,Lng
Origin,,,,
ABE,58.8,191.0,40.651940,-75.440560
ABI,98.2,28.0,32.411390,-99.681940
ABQ,34.0,1395.0,35.040278,-106.609167
ABR,65.1,62.0,45.449170,-98.421940
ABY,48.0,82.0,31.535560,-84.194440
ACT,58.3,104.0,31.611390,-97.230560
ACV,35.6,92.0,40.978060,-124.108610
ACY,33.6,341.0,39.457500,-74.577220
ADK,25.8,9.0,51.878060,-176.646110


In [39]:
import re

s = 'data/On_Time_On_Time_Performance_2017_9.zip'
r = re.search(r'(\d{4}_\d{1,2}).zip$', s)
r.group(1)

'2017_9'

In [4]:
import glob

i = 0
datapaths = glob.glob('data/On_Time_On_Time_Performance_*.zip')
for datapath in datapaths:
    i += 1
    date = re.search(r'(\d{4}_\d{1,2}).zip$', datapath).group(1)
    print("Working on file {} of {}: {}".format(i, len(datapaths), datapath))
    data = pd.read_csv(datapath)
    data = data.merge(airlines, how='left', left_on='AirlineID', right_on='Code', right_index=True)
    data = data[[
        'Year', 'Month', 'DayofMonth', 'DayOfWeek',
        'CarrierName', 'CarrierCode', 'FlightNum', 'TailNum',
        'Origin', 'Dest',
        'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime', 'ArrDelay',
        'CRSElapsedTime', 'ActualElapsedTime', 'Distance',
        'Cancelled', 'CancellationCode', 'Diverted',
        'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'
    ]]
    data['Flights'] = 1

    airport_delay_causes = data[['Origin', 'DayofMonth', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']].copy()
    airport_delay_causes['UnknownDelay'] = -data['DepDelayMinutes']
    airport_delay_causes = airport_delay_causes.set_index(['Origin', 'DayofMonth'])
    airport_delay_causes = airport_delay_causes.fillna(0)
    airport_delay_causes = airport_delay_causes.groupby(['Origin', 'DayofMonth']).sum()
    airport_delay_causes['UnknownDelay'] = -airport_delay_causes.sum(axis=1)
    airport_delay_causes['UnknownDelay'][airport_delay_causes['UnknownDelay'] < 0] = 0
    airport_delay_causes = airport_delay_causes.sort_index()
    delaypath = 'data/delay-causes_{}.csv'.format(date)
    print("Writing file to: ", delaypath)
    airport_delay_causes.to_csv(delaypath)

Working on file 1 of 21: data/On_Time_On_Time_Performance_2017_9.zip


/home/thierry/bin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Writing file to:  data/delay-causes_2017_9.csv
Working on file 2 of 21: data/On_Time_On_Time_Performance_2017_8.zip


/home/thierry/bin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Writing file to:  data/delay-causes_2017_8.csv
Working on file 3 of 21: data/On_Time_On_Time_Performance_2016_1.zip


...

Writing file to:  data/delay-causes_2016_11.csv
Working on file 7 of 21: data/On_Time_On_Time_Performance_2017_5.zip


/home/thierry/bin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (48,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Writing file to:  data/delay-causes_2017_5.csv
Working on file 8 of 21: data/On_Time_On_Time_Performance_2017_2.zip
Writing file to:  data/delay-causes_2017_2.csv
Working on file 9 of 21: data/On_Time_On_Time_Performance_2016_7.zip
Writing file to:  data/delay-causes_2016_7.csv
Working on file 10 of 21: data/On_Time_On_Time_Performance_2017_4.zip
Writing file to:  data/delay-causes_2017_4.csv
Working on file 11 of 21: data/On_Time_On_Time_Performance_2017_7.zip
Writing file to:  data/delay-causes_2017_7.csv
Working on file 12 of 21: data/On_Time_On_Time_Performance_2016_12.zip
Writing file to:  data/delay-causes_2016_12.csv
Working on file 13 of 21: data/On_Time_On_Time_Performance_2016_4.zip
Writing file to:  data/delay-causes_2016_4.csv
Working on file 14 of 21: data/On_Time_On_Time_Performance_2016_10.zip
Writing file to:  data/delay-causes_2016_10.csv
Working on file 15 of 21: data/On_Time_On_Time_Performance_2017_1.zip
Writing file to:  data/delay-causes_2017_1.csv
Working on file

/home/thierry/bin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Writing file to:  data/delay-causes_2016_5.csv
Working on file 19 of 21: data/On_Time_On_Time_Performance_2017_3.zip
Writing file to:  data/delay-causes_2017_3.csv
Working on file 20 of 21: data/On_Time_On_Time_Performance_2016_9.zip
Writing file to:  data/delay-causes_2016_9.csv
Working on file 21 of 21: data/On_Time_On_Time_Performance_2017_6.zip
Writing file to:  data/delay-causes_2017_6.csv


In [8]:
airlines = pd.read_csv('data/L_AIRLINE_ID.csv', index_col='Code')
airports = pd.read_csv('data/L_AIRPORT.csv', index_col='Code')

airlines_splitted = airlines.Description.apply(lambda d: d.split(': '))
airlines['CarrierName'] = airlines_splitted.apply(lambda d: d[0])
airlines['CarrierCode'] = airlines_splitted.apply(lambda d: d[-1])
airlines = airlines.drop('Description', axis=1)

airports_splitted = airports.Description.apply(lambda d: d.split(': '))
airports['AirportLocation'] = airports_splitted.apply(lambda d: d[0])
airports['AirportName'] = airports_splitted.apply(lambda d: d[-1])
airports = airports.merge(airports_loc, how='left', left_index=True, right_index=True)
airports = airports.drop('Description', axis=1)

airports

,AirportLocation,AirportName,Lat,Lng
Code,,,,
01A,"Afognak Lake, AK",Afognak Lake Airport,62.946389,-152.275833
03A,"Granite Mountain, AK",Bear Creek Mining Strip,65.548056,-161.071667
04A,"Lik, AK",Lik Mining Camp,NaN,NaN
05A,"Little Squaw, AK",Little Squaw Airport,NaN,NaN
06A,"Kizhuyak, AK",Kizhuyak Bay,NaN,NaN
07A,"Klawock, AK",Klawock Seaplane Base,NaN,NaN
08A,"Elizabeth Island, AK",Elizabeth Island Airport,NaN,NaN
09A,"Homer, AK",Augustin Island,NaN,NaN
1B1,"Hudson, NY",Columbia County,NaN,NaN


In [7]:
travel_base_url = "https://www.travelmath.com/airport/"
airport_coordinates_file = 'data/airport_coordinates.pickle'

import requests as rq
from bs4 import BeautifulSoup as bs
import pickle
import os

def get_coordinates_for_airport(airport_code):
    req = rq.get(travel_base_url+airport_code)
    cont = bs(req.text, 'html.parser')
    lat_long_elem = [ e for e in cont.select('div.leftcolumn p') if "Latitude/Longitude:" in e.text][0]
    lat_long = list(lat_long_elem.children)[1].split(",")
    lat, long = list(map(float, lat_long))
    return lat, long

def save_coords(airport_code, n):
    if n % 100 == 0:
        print("Airport {}".format(n))
    try:
        lat, long = get_coordinates_for_airport(airport_code)
        return airport_code, (lat, long)
    except Exception:
        return airport_code, None
    
if os.path.exists(airport_coordinates_file):
    with open(airport_coordinates_file, 'rb') as f:
        codes = pickle.load(f)
else:
    from concurrent import futures

    with futures.ProcessPoolExecutor(20) as executor:
        results = [executor.submit(save_coords, code, i) for i, code in enumerate(airports.index)]
        done, not_done = futures.wait(results)
    
    codes = {}

    for future in done:
        airport_code, long_lat = future.result()
        codes[airport_code] = long_lat

    with open(airport_coordinates_file, 'wb') as f:
        pickle.dump(codes, f)

In [92]:
# Download data zip from https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time
# Data exists from January 1987 to September 2017

from urllib.request import urlretrieve
from IPython.display import clear_output
from concurrent import futures
import os
import math
from time import sleep

transtats_base_download_url = "https://transtats.bts.gov/PREZIP/"
filename_template = "On_Time_On_Time_Performance_{year}_{month}.zip"

class DownloadHandler(object):
    def __init__(self, downloads):
        self.downloads = downloads
        self.status = {}
    
    def start(self, parallelism=None):
        print("Starting downloads...")
        with futures.ThreadPoolExecutor(parallelism) as executor:
            results = [executor.submit(self.download_routes_zip, *args) for args in self.downloads]
            while not self.done():
                self.print_status()
                sleep(5)
            done, not_done = futures.wait(results)
            print("All done!")
    
    def done(self):
        return all(map(lambda x: x[0] == 1, self.status.values()))
    
    def print_status(self):
        for filename, (percent, total) in self.status.items():
            print("{}: {:<30} [{}%, size={}]".format(filename, "#"*int(math.round(30*percent)), percent, total))
    
    def download_routes_zip(self, year, month):
        filename = filename_template.format(year=year, month=month)
        dl_url = transtats_base_download_url + filename
        dest_path = 'data/' + filename
        if os.path.exists(dest_path) and os.stat(dest_path).st_size > 0:
            self.status[filename] = (1, os.stat(dest_path).st_size)
        else:
            urlretrieve(dl_url, dest_path, reporthook=self.create_download_hook(filename))
            print("Downloaded: {}".format(filename))

    def create_download_hook(self, filename):
        def hook(downloaded_blocks, block_size, total_size):
            self.status[filename] = (min(downloaded_blocks * block_size / total_size, 1), total_size)
            print(self.status[filename])

to_download = []
for year in range(1987, 2018):
    for month in range(1, 13):
        if not (year == 2017 and month >= 10):
            to_download.append((year,month))
            
dh = DownloadHandler(to_download)
dh.start()

Starting downloads...
All done!


In [ ]:
routes = None

In [216]:
import zipfile
from io import BytesIO

year = 2016
for month in range(1,13):
    with zipfile.ZipFile('data/On_Time_On_Time_Performance_{year}_{month}.zip'.format(year=year, month=month), 'r') as z:
        for filename in z.namelist():
            if filename.endswith(".csv"): 
                df = pd.read_csv(BytesIO(z.read(filename)))
                new_routes = pd.DataFrame({'count': df.groupby(["OriginAirportID", "DestAirportID"]).size()})
                if routes is None:
                    routes = new_routes
                else:
                    routes = routes.add(new_routes, fill_value=0).astype(int) # type is converted to float when using add

/home/cranium/.anaconda3/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [241]:
import zipfile
from io import BytesIO

routes = None

# Sum of delays for year 2016 per airport, granularity of month
year = 2016
month = 10
for month in range(1,13):
    with zipfile.ZipFile('data/On_Time_On_Time_Performance_{year}_{month}.zip'.format(year=year, month=month), 'r') as z:
        for filename in z.namelist():
            if filename.endswith(".csv"): 
                df = pd.read_csv(BytesIO(z.read(filename)))
                per_airport = df.groupby(['Origin'])
                new_routes = pd.DataFrame({'flights': per_airport.size(), 'total_delays': per_airport['DepDelayMinutes'].sum()})
                new_routes = new_routes.set_index([new_routes.index, [month]*len(new_routes)])
                new_routes.index.names = ["Airport", "Month"]
                if routes is None:
                    routes = new_routes
                else:
                    routes = pd.concat([routes, new_routes])

/home/cranium/.anaconda3/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/cranium/.anaconda3/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/cranium/.anaconda3/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/cranium/.anaconda3/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or s

In [262]:
routes = routes.loc[routes.index.sort_values()]
with open('data/airport_delays_2016_months.json', 'w') as f:
    routes.to_csv(f, encoding='utf-8')

In [112]:
delay_columns = list(c for c in df.columns if "Delay" in c)
delay_columns

['DepDelay',
 'DepDelayMinutes',
 'DepartureDelayGroups',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrivalDelayGroups',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'DivArrDelay']